In [288]:
import gym

In [289]:
env = gym.make('Taxi-v2')

In [290]:
env.reset()

151

In [291]:
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [246]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

('initial observation code:', 354)
printing observation:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

('observations:', Discrete(500), 'n=', 500)
('actions:', Discrete(6), 'n=', 6)


In [295]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(4)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
('new observation code:', 251)
('reward:', -10)
('is game over?:', False)
printing new state:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)


In [293]:
env.action_space

Discrete(6)

In [173]:
env.observation_space

Discrete(500)

In [297]:
kkk = env.observation_space

In [314]:
new_obs

251

In [359]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(3)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
('new observation code:', 371)
('reward:', -1)
('is game over?:', False)
printing new state:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


In [ ]:
env.

In [134]:
action_to_i = {
    'down':0,
    'up':1,
    'right':2,
    'left':3,
    'pickup':4,
    'dropoff':5
}

In [135]:
k = env.reset()

In [ ]:
env.reset()

In [136]:
k

333

<TimeLimit<TaxiEnv<Taxi-v2>>>

In [444]:
def sample_reward(env, policy, t_max=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    iter_num = 0
    is_end = False
    while iter_num < t_max and is_end is False:
        cur_action = policy[s]
        s, reward, is_done, _ = env.step(cur_action)
        total_reward += reward
        iter_num += 1
        is_end = is_done
    return total_reward

In [445]:
import scipy.stats as sps

In [446]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return list(sps.binom(p=.5, n=5).rvs(size=500))

In [447]:
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    indices = sps.bernoulli(p).rvs(size=len(policy1))
    return [policy1[ind] if indices[ind] == 1 else policy2[ind] for ind in range(len(indices))]

In [448]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(get_random_policy(), policy, p)

In [449]:
jhgh = mutation(get_random_policy())

In [450]:
from tqdm import tqdm

In [451]:

n_epochs = 100 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick


In [452]:
from tqdm import tqdm

In [453]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for _ in range(n_times)]
    return float(np.mean(rewards))
        

In [454]:
print("initializing...")
pool = []
for i in tqdm(range(pool_size)):
    pool.append(get_random_policy())
pool_scores = [evaluate(cur_policy) for cur_policy in pool]




  0%|          | 0/100 [00:00<?, ?it/s]


 68%|██████▊   | 68/100 [00:00<00:00, 675.46it/s]


100%|██████████| 100/100 [00:00<00:00, 682.92it/s]

initializing...


In [455]:
import random

In [456]:
random.sample(range(pool_size), 2)

[93, 29]

In [457]:
#main loop
for epoch in tqdm(range(n_epochs)):
    print("Epoch %s:"%epoch)
    crossovered = []
    for i in range(n_crossovers):
        fst, snd = random.sample(range(pool_size), 2)
        crossovered.append(crossover(pool[fst], pool[snd]))
    mutated = []
    mutation_indices = random.sample(range(pool_size), n_mutations)
    for i in mutation_indices:
        mutated.append(mutation(pool[i]))
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    pool += mutated
    pool += crossovered
    pool_scores = [evaluate(cur_policy) for cur_policy in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])
#     print_policy(pool[-1])




  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0:







  1%|          | 1/100 [00:20<34:19, 20.81s/it]

('best score:', -251.56)
Epoch 1:





  2%|▏         | 2/100 [00:49<38:04, 23.31s/it]

...

('best score:', -100.0)
Epoch 98:





 99%|█████████▉| 99/100 [37:07<00:22, 22.87s/it]

('best score:', -100.0)
Epoch 99:





100%|██████████| 100/100 [37:30<00:00, 22.93s/it]

('best score:', -100.0)
